In [1]:
from src.preprocessing import process_text

def process_file_chunked(in_path: str, out_f, chunk_size: int = 10 * 1024 * 1024):
    """Process file in chunks, preserving article boundaries."""
    buffer = ""
    
    with open(in_path, 'r', encoding='utf-8') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            
            buffer += chunk
            
            # Find last complete article boundary (blank line)
            last_boundary = buffer.rfind('\n\n')
            
            if last_boundary != -1:
                # Process complete articles
                complete = buffer[:last_boundary]
                buffer = buffer[last_boundary + 2:]  # Keep incomplete part
                
                result = process_text(complete)
                if result:
                    out_f.write(result)
                    out_f.write('\n')
    
    # Process remaining buffer
    if buffer.strip():
        result = process_text(buffer)
        if result:
            out_f.write(result)
            out_f.write('\n')


# with open('merged.txt', 'w', encoding='utf-8') as out_f:
#     process_file_chunked('ubertext.social.filter_rus_gcld+short.text_only.txt', out_f)
#     process_file_chunked('ubertext.court.filter_rus_gcld+short.text_only.txt', out_f)

In [8]:
!./kenlm_bin//lmplz -o 1 < merged.txt > models/baseline1_cleaned_merged.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/nazara/UCU/NLP/ASSIGNMENT1/merged.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 272739121 types 1083989
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:13007868
Statistics:
1 1083989 D1=0.690469 D2=0.995374 D3+=1.23241
Memory estimate for binary LM:
type    MB
probing 45 assuming -p 1.5
probing 49 assuming -r models -p 1.5
trie    31 without quantization
trie    28 assuming -q 8 -b 8 quantization 
trie    31 assuming -a 22 array pointer compression
trie    28 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:13007868
=== 4/5 Calculating and writing order-interpolated probabilities ===
Chain sizes: 1:13007868
=== 5/5 Writing ARPA model ===
----5---10---

In [9]:
!./kenlm_bin//lmplz -o 2 < merged.txt > models/baseline2_cleaned_merged.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/nazara/UCU/NLP/ASSIGNMENT1/merged.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 272739121 types 1083989
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:13007868 2:26772203520
Statistics:
1 1083989 D1=0.718877 D2=1.01141 D3+=1.26483
2 15722553 D1=0.685865 D2=1.0965 D3+=1.37358
Memory estimate for binary LM:
type     MB
probing 296 assuming -p 1.5
probing 300 assuming -r models -p 1.5
trie    122 without quantization
trie     79 assuming -q 8 -b 8 quantization 
trie    122 assuming -a 22 array pointer compression
trie     79 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:13007868 2:251560848
----5---10---15---20---25---30---35---40---45---50---55---60

In [10]:
!./kenlm_bin//lmplz -o 3 < merged.txt > models/baseline3_cleaned_merged.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /home/nazara/UCU/NLP/ASSIGNMENT1/merged.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 272739121 types 1083989
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:13007868 2:9312070656 3:17460133888
Statistics:
1 1083989 D1=0.718877 D2=1.01141 D3+=1.26483
2 15722553 D1=0.75387 D2=1.10259 D3+=1.36763
3 45512017 D1=0.732599 D2=1.15699 D3+=1.38226
Memory estimate for binary LM:
type      MB
probing 1168 assuming -p 1.5
probing 1262 assuming -r models -p 1.5
trie     513 without quantization
trie     300 assuming -q 8 -b 8 quantization 
trie     480 assuming -a 22 array pointer compression
trie     267 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:13007868 2:2515

In [ ]:
import json 
from src.utils import process_dataset, evaluate_accuracy
from src.wrapper import get_model

samples = []
with open('ua_asr_hypotheses_500.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    result['reference'] = process_text(result['reference'])
    for i in range(len(result['hypotheses'])):
        result['hypotheses'][i] = process_text(result['hypotheses'][i])
    samples.append(result)



In [ ]:
import kenlm

model_paths = [
    'models/baseline1_cleaned_merged.arpa',
    'models/baseline2_cleaned_merged.arpa',
    'models/baseline3_cleaned_merged.arpa'
]
for path in model_paths:
    model = get_model(path)
    res = process_dataset(model,samples, use_perplexity=True)
    print(f'Evaluating model: {path}')
    accuracy = evaluate_accuracy(res, samples)
    print(f'Accuracy: {accuracy:.4f}')
    print('-----------------------')

Evaluating model: models/baseline1_cleaned_merged.arpa
Accuracy: 0.7020
-----------------------


Loading the LM will be faster if you build a binary file.
Reading /home/nazara/UCU/NLP/ASSIGNMENT1/models/baseline2_cleaned_merged.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.

Evaluating model: models/baseline2_cleaned_merged.arpa
Accuracy: 0.8240
-----------------------
Evaluating model: models/baseline3_cleaned_merged.arpa
Accuracy: 0.8420
-----------------------



Reading /home/nazara/UCU/NLP/ASSIGNMENT1/models/baseline3_cleaned_merged.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


# Why model fails?


In [ ]:
from src.utils import get_incorrect_samples

In [7]:
incorrect_samples = get_incorrect_samples(model, samples)

In [8]:
samples

[{'utt_id': 'ua_title_0000',
  'reference': 'вайлдер ортіс відео нокауту',
  'hypotheses': ['вайлдер ортіс відео нокауту',
   'вайлдер ортіз видео нокауту',
   'вайлдер ортіс видео нокоуту',
   'вайлдер вайлтер ортіс відео нокоуту нокауту',
   'войлтер ортіс відео накауту',
   'вайлдер ортіс вітео нокауту',
   'вайлдер ортіс відеа нокауту',
   'войлдер ортіз вітео нокауту']},
 {'utt_id': 'ua_title_0001',
  'reference': 'динамо підпише з родрігешем пятирічний контракт',
  'hypotheses': ['динамо підпише з родрігешем пятирічний контракт',
   'динамо підпише з родрігешем п ятирічний контрокт',
   'динамо підпише з родрікешем п ятирічний контракт',
   'динамо підпише з родрігешем п ятирічний контракт',
   'динамо підпише з родрігешем п ятирічний кондракт',
   'динамо підпише з родрігешем п ятирічней контракт',
   'динамо підпише з родрігешем п ятерічний кантракт',
   'динамо підпеше з родрігешем п ятирічний контракт']},
 {'utt_id': 'ua_title_0002',
  'reference': 'двох росіян визнали винним

In [9]:
sorted_incorrect = sorted(incorrect_samples, key=lambda x: x[1][1][1])

sorted_incorrect[1]


('бюджетnum\nшмигаль розповів що буде зі стипендіями зарплатами медиків і курсом долара',
 [('бюджет num\nшмигаль розповів що буде зі стипендіями зарплатами медиків курсом долара',
   110.37547557620223),
  ('бютжет num\nшмигаль розповів що буде зі стипендіями зарплатами медиків курсом долара',
   312.8185543280031),
  ('бюджетnum\nшмигаль розповів що буде зі стипендіями зарплатами медиків і курсом долара',
   383.2481262703121),
  ('бюджет num\nшмигаль розпавів що буде зі стипендіями зарплатами медиків курсом долара',
   519.3013497452654),
  ('бюджет num\nшмигаль разповів що буде зі стипендіями зарплатами медиків курсом долара',
   519.3013497452654),
  ('бюджет num\nшмигаль розповів ща буде зі стипендіями зарплатами медиків і курсом долара',
   1081.6081264450934),
  ('бюджет num\nшмигаль розповів що пуде зі стипендіями зарплатами медиків і курсом долара',
   1629.418382501189),
  ('бюджет num\nшмигаль розповів що буде зі стипендіямі зарплатами медиків і курсом долара',
   2552.8532

In [10]:
sorted_incorrect[2]

('наразі на апаратах швл знаходиться num хворих на covidnum українців',
 [('наразі на апаратах швл знаходиться num хворих на covid num українців',
   262.44559801514913),
  ('наразі на апаратах швл знаходиться num хворих на covidnum українців',
   338.2933352329838),
  ('наразі апаратах швл знаходиться num хворих на covid num українців',
   561.3348327188994),
  ('наразі на апаратах швл знахотиться num хворих на covid num українців',
   757.9981559410746),
  ('наразі но апаратах швл знаходиться num хворих на covid num українців',
   863.53922830124),
  ('наразі на апаратах швл знаходиться num хворих covid num українців',
   971.8366413977066),
  ('наразі на апаратах швл знаходиться num хвориг на covid num українців',
   2225.4668315929225),
  ('нарази на апаратах швл знаходидься num хворих на covid num українців',
   2875.5286101638026)])

In [11]:
sorted_incorrect[3]

('голова фонду держмайна сенниченко заявив про тиск з боку нардепаслуги холодова',
 [('голова фонду держмайна сенниченко заявив про тиск з боку нардепа слуги холодова',
   507.0782070787701),
  ('голова фонду держмайна сенниченко заявив заявив про тиск з боку нардепа слуги холодова',
   664.7103220240848),
  ('голова фонду держмайна сенниченко заявив про тиск з боку нардепаслуги холодова',
   823.6223733729305),
  ('голова фонду держмайна сенниченко саявив про тиск з боку нардепа слуги холодова',
   1471.8068242869922),
  ('голова фонду держмайна сенниченко зоявив про тиск з боку нардепа слуги холодова',
   1471.8068242869922),
  ('голова фонту держмайна сенниченко заявив про тиск з боку нардепа слуги холодова',
   4899.461547925687),
  ('голова фонду держмайна сенниченко заявив про тиск з богу нардиба слуги холодова',
   4924.701949668652),
  ('голова фонду держмайна сенниченко заявив про теск з боку нардепа слуги голодова',
   5005.021968418049)])

In [12]:
sorted_incorrect[4]

('вхід в громадські місця через qrкоди шмигаль розповів про num основних цифрових рішень',
 [('вхід в громадські місця через qr коди шмигаль розповів про num основних цифрових рішень',
   740.8425340148023),
  ('вхід в громадські місця через qrкоди шмигаль розповів про num основних цифрових рішень',
   1111.5190852294086),
  ('вхід громадські місця через qr коди шмигаль розповів про num основних цифрових рішень',
   1275.6193747251336),
  ('вхід в громадські місця через qr коди шмиголь розповів розповів про num основних цифрових рішень',
   1661.5499717720336),
  ('вхід в громадські мізця через qr коди шмигаль розповів про num основних цифрових рішень',
   2091.4061704008354),
  ('вхід громадські місця через qr коди шмиголь розповів про num основних цифрових рішень',
   2685.9703260203773),
  ('вхід в громадські місця через qr коди шмигаль розповів про num основних цифровех рішень',
   3496.0113550989436),
  ('вхід громадські місця через qr коди шмигаль розпавів розповів про num основн

In [13]:
sorted_incorrect[5]

('facebook блокує пости про користь вакцинації від covidnum',
 [('facebook блокує пости про користь вакцинації від covid num',
   1187.8897082198255),
  ('facebook блокує пости про користь вакцинації від covidnum',
   1470.0203636338288),
  ('facebook блокує пости про корість вакцинації від covid num',
   4096.302322188218),
  ('facebook блокує пости пра користь вакцинації від covid num',
   5382.23357841607),
  ('facebook блокує пости про користь вокцинації від covid num',
   7165.215748699486),
  ('facebook блокує пости про користь вакцинації віт covid num',
   10332.593013263177),
  ('facebook блокує пости бро користь вакцинациї від covid num',
   34873.99183328182),
  ('facebook блокує пості бро користь вакцинації вид covid num',
   42888.698602496326)])

In [14]:
sorted_incorrect[6]

('проросійського лідера опозиції молдови звільнили зпід варти',
 [('проросійського лідера опозиції молдови звільнили з під варти',
   1360.0009127083854),
  ('проросійського лідера опозиції молдови звільнили зпід варти',
   1752.5298454838085),
  ('проросійського лідера апозиції молдови звільнили з під варти',
   5925.315986216847),
  ('проросійського лідера опозиції молдови звильнили з під варти',
   6532.217180328292),
  ('проросійського лідера опозиції молдови звільнили з під варті',
   8855.38993038843),
  ('проросійського лідера опозиції молдови звільнили з під ворти',
   18193.182504292166),
  ('проросійського лідера опозиції молдови звільнили з під варди',
   18193.182504292166),
  ('проросійського лідира опозіції молдови звільнили з під варти',
   24973.808623538847)])

In [15]:
sorted_incorrect[7]

('uber на цьому тижні купить свого конкурента з дубая за numnum млрд',
 [('uber на цьому тижні купить свого конкурента з дубая за num num млрд',
   1840.5952982271406),
  ('uber на цьому тижні купить свого конкурента з дубая за numnum млрд',
   1971.4069383688368),
  ('uber на цьому тижні купить свого конкурента з дубая са num num млрд',
   2845.4185936041913),
  ('uber на цьому цьому тижні купить свого конкурента з дубая за num num млрд',
   4017.2258619329773),
  ('uber цьому тижні купить свого конкурента з дубая за num num млрд',
   5044.047110054329),
  ('uber на цьому тижні купить свого конкурента з дубая за num num млрт',
   6669.4621300496165),
  ('uber цьому тижні купить свого конкурента з дубая зо num num млрд',
   7558.281366056462),
  ('uber цьаму тижні купить свого конкурента з дубая за num num млрд',
   10744.615338541975)])

In [16]:
sorted_incorrect[8]

('майже num млн грн доходу\nтелекомоператор ахметова відзвітував за підсумками півріччя',
 [('майже num млн грн доходу\nтелеком оператор ахметова відзвітував за підсумками півріччя',
   1462.123533363946),
  ('майже num млн грн доходу\nтелекомоператор ахметова відзвітував за підсумками півріччя',
   2185.3598942629915),
  ('майже num млн грн дохоту\nтелеком оператор ахметова відзвітував за підсумками півріччя',
   3422.2665916059623),
  ('майже num млн грн доходу\nтелеком аператор ахметова відзвітував за підсумками півріччя',
   3559.615143120672),
  ('майже num млн грн доходу\nтелеком оператор ахметова відзвітував за підсумками бівріччя',
   4797.84098592603),
  ('майже num млн хрн доходу\nтелеком оператор ахметова відзвітував за підсумками півріччя',
   8327.90425256613),
  ('майже num млн грн доходу\nделеком оператор ахметова відзвітував за підзумкамі півріччя',
   14915.714632898298),
  ('майже num млн грн доходу\nтеликом оператор агметова відзвітував са підсумками півріччя',
   15

In [18]:
sorted_incorrect[-1]

('vici gaming чемпіон epicenter major',
 [('vici gaming чемпіон epicenter major major', 9498689.162492378),
  ('vici gaming чемпіон epicenter major', 12210751.72724596),
  ('vici gaming чемпіон чембіон epicenter major', 12958475.572435671),
  ('vici gaming чемпиан epicenter major', 27542393.479366038),
  ('vici gaming чембіон epicenter major', 27542393.479366038),
  ('vici gaming чимпіан epicenter major', 27542393.479366038),
  ('vici gaming чемпіан epicenter major', 27542393.479366038),
  ('vici gaming чемпион epicenter major', 27603425.637411177)])

In [19]:
sorted_incorrect[-2]

('андрій борячук угорці гидкі хлопці',
 [('андрій борячук угорці кидкі хлопці', 1627657.8247358466),
  ('андрій борячук угорці гидкі хлопці', 1631266.0417836236),
  ('андрій борячук угорці гидкі хлопці', 1631266.0417836236),
  ('андрій борячук угорці укорці гидкі хлопці', 2421249.594278617),
  ('андрій борячук укорці гидкі хлопці', 3061318.445686901),
  ('антрій борячук угорці гидкі хлопці', 10685519.07153601),
  ('андрий борячук угорці гидкі хлопці', 10709216.302144898),
  ('антрій борячук угорці гидкі хлобці', 52063901.863626555)])

In [30]:
! cat models/baseline2_cleaned_merged.arpa | grep "гидкі хлобц"

In [20]:
sorted_incorrect[-3]

('пивоварський пішов поєвропейськи карасьов',
 [('пивоварський пішов по європейське карасьов', 648419.5181208999),
  ('пиваварський пішов по європейське карасьов', 1114044.3125901155),
  ('пивоварський пішов по європейськи каросьов', 1988395.31558272),
  ('пивоварський пішов по європейськи каразьов', 1988395.31558272),
  ('пивоварський пішов по європейськи корасьов', 1988395.31558272),
  ('півоварський пішов по європейськи карасьов', 2182595.4026505887),
  ('певоварський пішов по європейськи карасьов', 2182595.4026505887),
  ('пивоварський пішов поєвропейськи карасьов', 7881525.0461821435)])

In [ ]:
! cat models/baseline2_cleaned_merged.arpa | grep "європейськи карасьов"

In [26]:
model = get_model('models/baseline1_cleaned_merged.arpa')


Found issues :

- issue with seleecting using logpob not perplexity (fixed)
- processing issue. 
- different distributions, OOD samples (huge perplexity), some samples do not have 2grams, 3grams present in training data.